## imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime
from pyspark.sql.window import Window

## path

In [0]:
%run ./00_setup

## load silver clean data

In [0]:
df_silver = spark.read.format("delta").load(silver_write_path)

display(df_silver.limit(5))

PULocationID,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,_file_path,ingest_time,trip_pickup_datetime,trip_dropoff_datetime,trip_duration,pickup_date,pickup_hour,pickup_borough,pickup_zone_name
164,2,1,1.59,1.0,N,48,1,13.5,1.0,0.5,1.5,0.0,1.0,20.0,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:52.447Z,2022-12-29T00:00:08.000Z,2022-12-29T00:13:18.000Z,13.166666666666666,2022-12-29,0,Manhattan,Midtown South
246,2,2,1.48,1.0,N,158,1,9.3,1.0,0.5,1.0,0.0,1.0,15.3,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:52.447Z,2022-12-29T00:01:16.000Z,2022-12-29T00:08:08.000Z,6.866666666666666,2022-12-29,0,Manhattan,West Chelsea/Hudson Yards
162,2,1,2.64,1.0,N,68,1,12.8,1.0,0.5,3.56,0.0,1.0,21.36,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:52.447Z,2022-12-29T00:04:05.000Z,2022-12-29T00:12:22.000Z,8.283333333333333,2022-12-29,0,Manhattan,Midtown East
48,2,1,1.82,1.0,N,143,1,12.1,1.0,0.5,4.28,0.0,1.0,21.38,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:52.447Z,2022-12-29T00:00:30.000Z,2022-12-29T00:11:19.000Z,10.816666666666666,2022-12-29,0,Manhattan,Clinton East
132,2,2,19.71,2.0,N,87,1,70.0,0.0,0.5,15.05,0.0,1.0,90.3,2.5,1.25,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:52.447Z,2022-12-29T00:01:40.000Z,2022-12-29T00:36:13.000Z,34.55,2022-12-29,0,Queens,JFK Airport


## daily trips per zone

In [0]:
df_daily_trip = df_silver.groupBy('pickup_date','PULocationID','pickup_zone_name')  \
    .agg(count('*').alias('total_trips'),
         avg('trip_distance').alias('avg_trip_distance'),
         avg('fare_amount').alias('avg_fare'),
         sum('total_amount').alias('total_revenue'),
         expr("percentile_approx(tip_amount, 0.5)").alias("median_tip")
        )   \
    .orderBy('pickup_date','PULocationID')
display(df_daily_trip.limit(20))

pickup_date,PULocationID,pickup_zone_name,total_trips,avg_trip_distance,avg_fare,total_revenue,median_tip
2022-11-30,48,Clinton East,2,2.355,9.75,32.519999999999996,2.16
2022-11-30,79,East Village,5,3.364,12.5,96.36,2.76
2022-11-30,90,Flatiron,2,1.8,8.0,27.76,2.0
2022-11-30,100,Garment District,2,4.045,14.5,41.760000000000005,1.0
2022-11-30,107,Gramercy,1,1.46,6.5,10.3,0.0
2022-11-30,114,Greenwich Village South,1,1.03,5.5,10.97,1.67
2022-11-30,132,JFK Airport,3,13.11,39.333333333333336,164.26999999999998,11.31
2022-11-30,138,LaGuardia Airport,1,8.93,27.0,50.18,11.58
2022-11-30,141,Lenox Hill West,1,3.12,12.5,19.56,3.26
2022-11-30,142,Lincoln Square East,1,2.84,12.0,18.96,3.16


In [0]:
df_daily_trip.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(gold_write_path + "/daily")

## hourly trip per zone

In [0]:
df_hourly_trip = df_silver.groupBy('pickup_date','pickup_hour','PULocationID','pickup_zone_name')  \
    .agg(count('*').alias('total_trips'),
         avg('fare_amount').alias('avg_fare')) \
    .orderBy("pickup_date","pickup_hour","PULocationID")

display(df_hourly_trip.limit(20))

pickup_date,pickup_hour,PULocationID,pickup_zone_name,total_trips,avg_fare
2022-11-30,19,107,Gramercy,1,6.5
2022-11-30,19,132,JFK Airport,2,33.0
2022-11-30,22,249,West Village,1,28.5
2022-11-30,23,48,Clinton East,2,9.75
2022-11-30,23,79,East Village,5,12.5
2022-11-30,23,90,Flatiron,2,8.0
2022-11-30,23,100,Garment District,2,14.5
2022-11-30,23,114,Greenwich Village South,1,5.5
2022-11-30,23,132,JFK Airport,1,52.0
2022-11-30,23,138,LaGuardia Airport,1,27.0


In [0]:
df_hourly_trip.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(gold_write_path +"/hourly")

## top zone per date

In [0]:
top_zones_per_day = (
    df_silver.groupBy("pickup_date", "PULocationID", "pickup_zone_name")
          .agg(count("*").alias("total_trips"))
          .withColumn("rank", dense_rank().over(
              Window.partitionBy("pickup_date")
                    .orderBy(desc("total_trips"))
          ))
          .filter("rank <= 5")   # top 5
          .orderBy("pickup_date", "rank")
)

In [0]:

top_zones_per_day.write.format("delta") \
    .mode("overwrite") \
    .save(gold_write_path + "/top_zones_per_day")

## revenue by borough

In [0]:
revenue_by_borough = (
    df_silver.groupBy("pickup_date", "pickup_borough")
          .agg(
              sum("total_amount").alias("daily_revenue"),
              count("*").alias("total_trips"),
              avg("total_amount").alias("avg_revenue_per_trip")
          )
          .orderBy("pickup_date", "pickup_borough")
)



In [0]:
revenue_by_borough.write.format("delta") \
    .mode("overwrite") \
    .save(gold_write_path + "/revenue_by_borough")

## average fare trends

In [0]:
avg_fare_trend = (
    df_silver.groupBy("pickup_date")
          .agg(
              avg("fare_amount").alias("avg_fare"),
              avg("total_amount").alias("avg_total")
          )
          .orderBy("pickup_date")
)



In [0]:
avg_fare_trend.write.format("delta") \
    .mode("overwrite") \
    .save(gold_write_path + "/avg_fare_trend")

## load gold dataset

In [0]:
daily_trip = spark.read.format('delta').load(gold_write_path + "/daily")
hourly_trip = spark.read.format('delta').load(gold_write_path + "/hourly")
top_5_zone_trip = spark.read.format('delta').load(gold_write_path + "/top_zones_per_day")
borough_revenue = spark.read.format('delta').load(gold_write_path + "/revenue_by_borough")
avg_fare = spark.read.format('delta').load(gold_write_path + "/avg_fare_trend")

# daily_trip.limit(5).show()
# hourly_trip.limit(5).show()
# top_5_zone_trip.limit(5).show()
# borough_revenue.limit(5).show()
# avg_fare.limit(5).show()

daily_trip.limit(5).display()
hourly_trip.limit(5).display()
top_5_zone_trip.limit(5).display()
borough_revenue.limit(5).display()
avg_fare.limit(5).display()

pickup_date,PULocationID,pickup_zone_name,total_trips,avg_trip_distance,avg_fare,total_revenue,median_tip
2022-11-30,48,Clinton East,2,2.355,9.75,32.519999999999996,2.16
2022-11-30,79,East Village,5,3.364,12.5,96.36,2.76
2022-11-30,90,Flatiron,2,1.8,8.0,27.76,2.0
2022-11-30,100,Garment District,2,4.045,14.5,41.760000000000005,1.0
2022-11-30,107,Gramercy,1,1.46,6.5,10.3,0.0


pickup_date,pickup_hour,PULocationID,pickup_zone_name,total_trips,avg_fare
2022-11-30,19,107,Gramercy,1,6.5
2022-11-30,19,132,JFK Airport,2,33.0
2022-11-30,22,249,West Village,1,28.5
2022-11-30,23,48,Clinton East,2,9.75
2022-11-30,23,79,East Village,5,12.5


pickup_date,PULocationID,pickup_zone_name,total_trips,rank
2022-11-30,161,Midtown Center,6,1
2022-11-30,79,East Village,5,2
2022-11-30,163,Midtown North,4,3
2022-11-30,132,JFK Airport,3,4
2022-11-30,90,Flatiron,2,5


pickup_date,pickup_borough,daily_revenue,total_trips,avg_revenue_per_trip
2022-11-30,Manhattan,688.6499999999997,35,19.675714285714278
2022-11-30,Queens,214.45,4,53.6125
2022-11-30,Unknown,16.56,1,16.56
2022-12-01,Bronx,5134.290000000004,159,32.29113207547172
2022-12-01,Brooklyn,18516.139999999927,637,29.067723704866445


pickup_date,avg_fare,avg_total
2022-11-30,15.3875,22.991499999999995
2022-12-01,15.768457667305292,23.034177134684988
2022-12-02,15.096530191969485,22.19291985891723
2022-12-03,14.475582420370317,21.003538994498154
2022-12-04,16.157855711791317,23.295754434338498
